#Fishers Exact Test

In [ ]:
import pandas as pd
import numpy as np
import openpyxl

import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

In [ ]:
info=pd.read_csv('/koreamerged0710_ccRCC_2panelnanremove.csv')#koreamerged0710_ccRCC_2panelnanremove.csv
info

In [ ]:
gdata=info.iloc[:,15:]
cdata=info.iloc[:,:15]
info = info.rename(columns={'Sex (F/M)': 'Sex'})
cdata = cdata.rename(columns={'Sex (F/M)': 'Sex'})

In [ ]:
import pandas as pd
import math
from scipy.stats import fisher_exact


# Create an ExcelWriter object to save the results
output_filename = 'Fisher_67_0705_CI.xlsx'
excel_writer = pd.ExcelWriter(output_filename, engine='xlsxwriter')

for f_cli in cdata.columns[1:]:
    results = []  # 결과를 저장할 리스트

    for g_cli in gdata.columns:
        try:
            # 작업할 열 추출
            c, g = f_cli, g_cli
            filter_data = info[[c, g]]
            cross_table = pd.crosstab(filter_data[c], filter_data[g], rownames=[c], colnames=[g])

            # 2x1 분할표일 경우 0으로 채우기
            if len(cross_table.columns) == 1:
                if cross_table.columns[0] == 1.0:
                    cross_table[0.0] = 0
                elif cross_table.columns[0] == 0.0:
                    cross_table[1.0] = 0

            print(cross_table)

            # Get row count
            row_count = cross_table.sum(axis=1)

            # # Check table shape
            # if cross_table.shape != (2, 2):
            #     raise ValueError("The input `cross_table` must be of shape (2, 2).")

            # Fisher's exact test 수행
            odds_ratio, p_value = fisher_exact(cross_table)

            # 결과 출력
            print("Odds Ratio:", odds_ratio)
            print("p-value:", p_value)

            # Calculate confidence interval for odds ratio
            se = math.sqrt(1 / row_count[0] + 1 / row_count[1])
            lower_ci = math.exp(math.log(odds_ratio) - (1.96 * se))
            upper_ci = math.exp(math.log(odds_ratio) + (1.96 * se))
            print("ci: ",lower_ci,'~',lower_ci)
            # 결과를 딕셔너리로 저장
            result = {
                'f_cli': f_cli,
                'g_cli': g_cli,
                'cross_table': cross_table,
                '0.0':cross_table.iloc[0,0],
                '0.1':cross_table.iloc[0,1],
                '1.0':cross_table.iloc[1,0],
                '1.1':cross_table.iloc[1,1],
                'row_count': row_count,
                'odds_ratio': odds_ratio,
                'odds_ratio_ci': (lower_ci, upper_ci),
                'p_value': p_value
            }
            results.append(result)

        except ValueError as ve:
            print("Error occurred:", str(ve))
            # # 2x2 분할표가 아닌 경우 빈 부분만 0으로 채워넣기
            cross_table = cross_table.reindex(columns=[0, 1], fill_value=0)
            row_count = cross_table.sum(axis=1)
            odds_ratio = 0
            lower_ci = 0
            upper_ci = 0
            p_value = 0

            # 결과를 딕셔너리로 저장
            result = {
                'f_cli': f_cli,
                'g_cli': g_cli,
                'cross_table': cross_table,
                'row_count': row_count,
                'odds_ratio': odds_ratio,
                'odds_ratio_ci': (lower_ci, upper_ci),
                'p_value': p_value
            }
            results.append(result)
    # 결과를 데이터프레임으로 변환
    result_df = pd.DataFrame(results)
    result_df.to_excel(excel_writer, sheet_name=f_cli, index=False)
    if(f_cli=='Sex'):
       break

# 엑셀로 저장
excel_writer.close()
print("Results saved to", output_filename)

#Kaplan meier

In [ ]:
# Filter the DataFrame for each gender
male_df = info[info['donor_sex'] == 'male']
female_df = info[info['donor_sex'] == 'female']

# Create new DataFrames for each gender
male_dataframe = pd.DataFrame(male_df)
female_dataframe = pd.DataFrame(female_df)

In [ ]:
pat_cli_f = female_dataframe.loc[:, ['icgc_donor_id', 'submitted_donor_id', 'recurrence','death',
                                   'donor_sex', 'sex','donor_relapse_interval','donor_survival_time',]]
gene_f = female_dataframe.loc[:, ['icgc_donor_id'] + list(female_dataframe.columns[13:])]

In [ ]:
pat_clim = pd.merge(pat_cli_f, gene_f, on='icgc_donor_id', how='inner')
genes=gene_f.columns[1:]

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['legend.title_fontsize'] = 'medium'

In [ ]:
def get_pval_from_logrank_test(data, survival_type, survival_value):
  pvals = []

#  if survival_type == 'OS, day':
#      data = data.dropna()
#      data = data.reset_index(drop=True)

  for gene in genes:

    with_mutation = data[data[gene] == 1]
    without_mutation = data[data[gene] == 0]

    T = without_mutation[f'{survival_type}']
    E = without_mutation[survival_value]
    T1 = with_mutation[f'{survival_type}']
    E1 = with_mutation[survival_value]

    result = logrank_test(T, T1,event_observed_A=E, event_observed_B=E1)
    pvals.append([gene, result.p_value, len(T), len(T1)])

    if result.p_value > 0:
      print(gene)
      print(result.p_value)
      plt.figure(figsize=(10, 8))
      kmf = KaplanMeierFitter()
      kmf.fit(durations =  with_mutation[f'{survival_type}'], event_observed = with_mutation[survival_value] , label=f"Altered group")#
      ax_kmf = kmf.plot(ci_show=False, c='crimson')
      kmf.fit(durations =  without_mutation[f'{survival_type}'], event_observed = without_mutation[survival_value] , label=f"Unaltered group")#
      ax_kmf = kmf.plot(ax=ax_kmf, ci_show=False, c='royalblue')
      ax_kmf.set_title(f'{gene}', fontsize=15, style='italic', pad=30)
      ax_kmf.set_xlabel(f'Month', fontsize=12)
      # ax_kmf.set_ylabel(f'{survival_type} probability' , fontsize=20)
      ax_kmf.legend(prop={'size':12}, frameon=False)
      plt.xlim(left=0)
      plt.ylim(bottom=0)
      plt.xticks(np.arange(0,21,5),fontsize=12)
      plt.yticks(fontsize=12)
      plt.text(15, 0.80, 'p = '+ str(round(result.p_value, 10)), fontsize = 12, style='italic')

      #plt.savefig(f'{survival_type}-{gene}_male.png')
      #files.download(f'{survival_type}-{gene}.png')

  return pvals

In [ ]:
pat_clim = pat_clim.dropna(subset=['death'])
pat_clim = pat_clim.dropna(subset=['donor_survival_time'])
logrank_result = get_pval_from_logrank_test(pat_cli,'donor_survival_time','death')#pat_clim#mtf
print(logrank_result)
pd.DataFrame(logrank_result).to_csv('ccRCC_survival-RECA_Logrank_total_OS.csv', index=False,
               header=["gene","p_value(mut-with+withpout)","without_mutation","with_mutation"])

#collective

In [ ]:
pat_cli=pd.read_csv(dir+'/koreamerged0705_ccRCCv3.csv')

In [ ]:
pat_cli_f=male_dataframe.loc[:,['SAMPLE','Age','Sex', 'Nuclear Grade',
       'Sarcomatoid component','T-Stage', 'TNM N-Stage', 'TNM M-Stage',
       'recurrence','death','Overall Survival (Month)', 'Disease Free Survival (Month)']]
gene_f=male_dataframe.loc[:,male_dataframe.columns[16:]]
gene_f.head()

In [ ]:
# OS_col=['ACTR1B', 'BPNT1', 'BRD4', 'BZRAP1', 'C15orf27', 'C16orf72', 'CD1C', 'CEP128', 'CIITA',
#         'COL5A1', 'CYP51A1', 'DNAAF2', 'EPHB1', 'ESRP2', 'FBXW9', 'ITGA3', 'ITGA4', 'ITGA8', 'KIF5C',
#         'KPRP', 'MAOB', 'MUC17', 'MYH10', 'OGFR', 'OR1S1', 'PCBP4', 'PCGF2', 'PCSK2', 'PLEKHB2', 'PPM1F',
#         'RAB40B', 'RRP36', 'RTL1', 'RYR1', 'SMARCA1', 'SNX7', 'SSX2IP', 'TAS1R2', 'THUMPD2', 'VPS13D']
OS_col = ["ASXL3", "HAUS7", "NBPF10"]

DFS_col = ["ACSS3", "BAP1", "CFP", "FAM47A", "NCOR1P1", "SCRN1"]

# OS_col3=['BAP1','PCSK2','SPATA13']
# DFS_col3=['BAP1','PCSK2','SPATA13']

In [ ]:
gene_f=gene_f.loc[:,OS_col+DFS_col]#,"CISD3","CFDP1"
mtf=pat_cli.loc[:,['SAMPLE','recurrence', 'death','Overall Survival (Months)', 'Disease Free (Months)',]+(OS_col)+(DFS_col)]#
genes=gene_f.columns

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['legend.title_fontsize'] = 'medium'

In [ ]:
with_mu="ACTR1B == 1. or BRD4 == 1. or EPHB1 == 1 or FBXW9 == 1. or ITGA3 == 1. or ITGA8 == 1. or KPRP == 1. or MAOB == 1. or MUC17 == 1. or MYH10 == 1. or OR1S1 == 1. or SMARCA1 == 1. or SSX2IP == 1. or TAS1R2 == 1. or THUMPD2 == 1."
without_mu="ACTR1B == 0 and BRD4 == 0 and EPHB1 == 0 and FBXW9 == 0 and ITGA3 == 0 and ITGA8 == 0 and KPRP == 0 and MAOB == 0 and MUC17 == 0 and MYH10 == 0 and OR1S1 == 0 and RAB40B == 0 and SMARCA1 == 0 and SSX2IP == 0 and TAS1R2 == 0 and THUMPD2 == 0"

In [ ]:
without_mutation = mtf.query(without_mu)#
without_mutation

In [ ]:
def cget_pval_from_logrank_test(data, survival_type, survival_value, name):
  pvals = []

  if survival_type == 'Disease Free (Months)':
      data = data.dropna()
      data = data.reset_index(drop=True)

  with_mutation = data.query(with_mu)# ,"CISD3","CFDP1"
  without_mutation = data.query(without_mu)#
  print(with_mutation.index)
  print(without_mutation.index)

  T = without_mutation[f'{survival_type}']
  E = without_mutation[survival_value]
  T1 = with_mutation[f'{survival_type}']
  E1 = with_mutation[survival_value]

  result = logrank_test(T, T1, event_observed_A=E, event_observed_B=E1)
  print("result : ",result)
  pvals.append([name, result.p_value, len(T), len(T1), len(without_mutation[without_mutation[survival_value] == 1]), len(with_mutation[with_mutation[survival_value] == 1])])
  print("pvals : ",pvals,"//",result.p_value)

  if result.p_value > 0:
    #print(T, E, T1, E1)
    print("T",T)
    print("E",E)
    plt.figure(figsize=(10, 8))
    kmf = KaplanMeierFitter()
    kmf.fit(durations =  with_mutation[f'{survival_type}'], event_observed = with_mutation[survival_value] , label=f"Altered group")
    ax_kmf = kmf.plot(ci_show=False, c='crimson')
    kmf.fit(durations =  without_mutation[f'{survival_type}'], event_observed = without_mutation[survival_value] , label=f"Unaltered group")
    ax_kmf = kmf.plot(ax=ax_kmf, ci_show=False, c='royalblue')
    ax_kmf.set_title(f'{name}', fontsize=30, style='italic', pad=30)
    ax_kmf.set_xlabel(f'{survival_type}_survival' , fontsize=15)
    # ax_kmf.set_ylabel(f'{survival_type} probability' , fontsize=20)
    ax_kmf.legend(prop={'size':10}, frameon=False)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.text(60, 0.90, 'p = '+ str(round(result.p_value,40)), fontsize = 15, style='italic')

    #plt.savefig(f'{survival_type}-{name}_.png')
    # files.download(f'{survival_type}-{name}.png')

  return pvals

In [ ]:
# logrank_result = cget_pval_from_logrank_test(mtf,['OS','DFS'],['Death','Recurrence'])

#logrank_result = cget_pval_from_logrank_test(mtf,'Overall Survival (Months)','death',"OS_TCGA_Collective")
logrank_result = cget_pval_from_logrank_test(mtf,'Disease Free (Months)','recurrence',"DFS_TCGA_Collective")
print("logrank_result\n",logrank_result)
# pd.DataFrame(logrank_result).to_csv('PRCC_survival-KIRP_Logrank_DFS_0609.csv', index=False
#             ,header=["gene","p_value(mut-with+withpout)","without_mutation","with_mutation","without_mutation_Altercount","with_mutation_Altercount"])